#Web scrapping

In [ ]:
!pip install playwright
!playwright install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.9/539.9 kB 47.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 2

In [ ]:
# łączenie się ze stroną internetową Hebe dla kategorii 'szampony'
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

urls = set()

async def run():
    previous_length = len(urls)
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto("https://www.hebe.pl/pielegnacja-szampony/")
        while True:
          await page.click('text=ZOBACZ WIĘCEJ')
          # Pobieranie zawartości strony
          content = await page.content()
          # Scrapowanie przy użyciu parsera BeautifulSoup
          soup = BeautifulSoup(content, 'html.parser')
          # Wyszukiwanie kodów, zawierających fragment stron internetowych poszczególnych produktów
          elements = soup.find_all("div", class_="thumb-link")
          urls.update([link['data-href'] for link in elements])
          print(len(urls))
          if len(urls) != previous_length:
            previous_length = len(urls)
          else:
            break
        await browser.close()  

await run()        

In [ ]:
# utworzenie listy z adresami do poszczególnych produktów
root = 'https://www.hebe.pl'
web_pages = []
for url in urls:
  web_page = root + url
  web_pages.append(web_page)     

In [ ]:
len(web_pages)

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# pobieranie danych o nazwach produktów
names=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  name = soup.findAll('div', attrs={'class':'product-summary__title'})
  names.append(name[-1].text)

In [ ]:
# pobieranie danych o składnikach produktów
ingredients=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  data = soup.findAll('div', attrs={'class':'ui-expandable__inner'})
  ingredients.append(data[-1].text)

In [ ]:
# pobieranie danych o markach produktów
brand_names=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  brand = soup.findAll('a', attrs={'class':'product-content__link'})
  if len(brand):
    brand_names.append(brand[0].text)
  else:
    brand_names.append('brak')

In [ ]:
# pobieranie danych o ocenach produktów
rates=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  rate = soup.findAll("div", class_="reviews-stars__number reviews-stars__number--first")
  if len(rate):
    rates.append(rate[0].text)
  else:
    rates.append('brak oceny')

In [ ]:
#pobieranie danych o cenach produktów
prices=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  price = soup.findAll('span', attrs={'class':'price-product__amount'})
  prices.append(price[-1].text)

prices_decimal=[]

for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  decimal = soup.findAll('span', attrs={'class':'price-product__currency'})
  prices_decimal.append(decimal[-1].text)

In [ ]:
# pobieranie danych o ilościach opinii
reviews=[]
for item in web_pages:
  page = requests.get(item)
  soup = BeautifulSoup(page.content, "html.parser")
  review = soup.findAll("div", class_="reviews-stars__count link link--underline link--bold link--hover")
  if len(review):
    reviews.append(review[0].text)
  else:
    reviews.append('brak')

In [ ]:
import pandas as pd

In [ ]:
# stworzenie DataFrame z pobranych danych
df = pd.DataFrame({'ProductName': names, 'Ingredients': ingredients, 'Rate': rates, 'Price': prices, 'Price_decimal': prices_decimal, 'Review': reviews, 'BrandName': brand_names})

In [ ]:
# pobranie DataFrame jako plik csv
df.to_csv('products.csv', index=False, encoding='utf-8')